References :<br>
https://www.youtube.com/watch?v=6zm9NC9uRkk <br>
https://github.com/skipgram/modern-nlp-in-python-2019/blob/master/notebooks/Modern_NLP_in_Python.ipynb

In [1]:
import os
data_directory = os.path.join('.','yelp_dataset')

businesses_filepath = os.path.join(data_directory, 'business.json')

with open(businesses_filepath) as f:
    first_business_record = f.readline() 
    
print(first_business_record)

{"business_id":"f9NumwFMBDn751xgFiRbNA","name":"The Range At Lake Norman","address":"10913 Bailey Rd","city":"Cornelius","state":"NC","postal_code":"28031","latitude":35.4627242,"longitude":-80.8526119,"stars":3.5,"review_count":36,"is_open":1,"attributes":{"BusinessAcceptsCreditCards":"True","BikeParking":"True","GoodForKids":"False","BusinessParking":"{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}","ByAppointmentOnly":"False","RestaurantsPriceRange2":"3"},"categories":"Active Life, Gun\/Rifle Ranges, Guns & Ammo, Shopping","hours":{"Monday":"10:0-18:0","Tuesday":"11:0-20:0","Wednesday":"10:0-18:0","Thursday":"11:0-20:0","Friday":"11:0-20:0","Saturday":"11:0-20:0","Sunday":"13:0-18:0"}}



In [2]:
#Reviews

review_path = os.path.join(data_directory,'review.json')

with open(review_path) as f:
    first_review = f.readline()
    
print(first_review)

{"review_id":"xQY8N_XvtGbearJ5X4QryQ","user_id":"OwjRMXRC0KyPrIlcjaXeFQ","business_id":"-MhfebM0QIsKt87iDN-FNw","stars":2.0,"useful":5,"funny":0,"cool":0,"text":"As someone who has worked with many museums, I was eager to visit this gallery on my most recent trip to Las Vegas. When I saw they would be showing infamous eggs of the House of Faberge from the Virginia Museum of Fine Arts (VMFA), I knew I had to go!\n\nTucked away near the gelateria and the garden, the Gallery is pretty much hidden from view. It's what real estate agents would call \"cozy\" or \"charming\" - basically any euphemism for small.\n\nThat being said, you can still see wonderful art at a gallery of any size, so why the two *s you ask? Let me tell you:\n\n* pricing for this, while relatively inexpensive for a Las Vegas attraction, is completely over the top. For the space and the amount of art you can fit in there, it is a bit much.\n* it's not kid friendly at all. Seriously, don't bring them.\n* the security is n

In [3]:
#Extract all business ids
import json

restaurant_ids=set()

with open(businesses_filepath) as f:
    
    for business_record in f:
        business = json.loads(business_record)
        
        if not business.get('categories'):
            continue
        
        if 'Restaurants' not in business['categories']:
            continue
            

        restaurant_ids.add(business['business_id'])
        
restaurant_ids = frozenset(restaurant_ids)

print(f'{len(restaurant_ids)} Restaurants in Dataset')

63944 Restaurants in Dataset


In [8]:
#Create a cratch directory to store all reviews
scratch_path = os.path.join('.','scratch')

In [9]:
try:
    os.mkdir(scratch_path)
except FileExistsError:
    pass

review_txt_filepath = os.path.join(scratch_path,'review_text_all.txt')

In [5]:
#Extract all reviews snd store it in the new file created

execute = False

review_count = 0

if execute:
    
    with open(review_txt_filepath,"w") as review_txt_file:
        
        with open(review_path) as review_file:
            
            for review_json in review_file:
                review = json.loads(review_json)
                
                if review['business_id'] not in restaurant_ids:
                    continue
                    
                review_txt_file.write(review['text'].replace("\n","\\n")+"\n")
                review_count+=1
                
    print(f'Text from {review_count:,} restaurant reviews written to the new txt file.')

else:
    
    # count the reviews in the existing file
    with open(review_txt_filepath) as review_txt_file:
        for review_count, line in enumerate(review_txt_file):
            pass
        
    print(f'Text from {review_count + 1:,} restaurant reviews in the txt file.')
                
                


Text from 5,058,162 restaurant reviews in the txt file.


In [4]:
import spacy
from spacy import displacy
import pandas as pd
import itertools as it

nlp = spacy.load('en_core_web_sm')

In [7]:
#Length of each review
review_dict={}
with open(review_txt_filepath) as review_txt_file:
    review_dict = {review_count:len(line) for review_count,line in enumerate(review_txt_file)}
        
    

In [9]:
max_key = max(review_dict,key=review_dict.get)
print(max_key)

1788257


In [58]:
#Sample Review
#10 14 15 22
review_num = 22
with open(review_txt_filepath) as f:
    sample_review = list(it.islice(f, review_num, review_num+1))[0]
    sample_review = sample_review.replace('\\n', '\n')
        
print(sample_review)

This actually used to be one of my favorite hotel/casinos back in the day.  Oh, how things have changed.

There are some goods, however.  If you are a conventioneer, you have instant and direct walking access to the main convention center.  But don't think about parking here for said conventions.  $30 a day for parking, Hilton?  Seriously?  That's robbery.  And greed.

The biggest disappointment is the killing off of Star Trek: The Experience over two years ago.  Despite this place being off-Strip, I made it a point to go here because they had such a fun and unique attraction here.  It would be one thing if they had something in mind to replace it, but either poor planning or the economy.... It's dead, Jim. 

The "Space Lounge" is still there.  An entire casino area with a heavy sci-fi theme to it.  Except there's no casino here.  It's a huge space themed area with a ghost town bar, no slots, a few couches, a makeshift stage, beer signs, and a few remnants of Star Trek themes.  All of 

In [59]:
%%time
parsed_view = nlp(sample_review)

CPU times: user 267 ms, sys: 25.2 ms, total: 292 ms
Wall time: 160 ms


In [60]:
parsed_view = nlp(sample_review)

In [61]:
print(parsed_view)

This actually used to be one of my favorite hotel/casinos back in the day.  Oh, how things have changed.

There are some goods, however.  If you are a conventioneer, you have instant and direct walking access to the main convention center.  But don't think about parking here for said conventions.  $30 a day for parking, Hilton?  Seriously?  That's robbery.  And greed.

The biggest disappointment is the killing off of Star Trek: The Experience over two years ago.  Despite this place being off-Strip, I made it a point to go here because they had such a fun and unique attraction here.  It would be one thing if they had something in mind to replace it, but either poor planning or the economy.... It's dead, Jim. 

The "Space Lounge" is still there.  An entire casino area with a heavy sci-fi theme to it.  Except there's no casino here.  It's a huge space themed area with a ghost town bar, no slots, a few couches, a makeshift stage, beer signs, and a few remnants of Star Trek themes.  All of 

In [63]:
for num,sentence in enumerate(parsed_view.sents):
    print(f'Sentence {num+1}')
    print(sentence)
    print('')

Sentence 1
This actually used to be one of my favorite hotel/casinos back in the day.  

Sentence 2
Oh, how things have changed.



Sentence 3
There are some goods, however.  

Sentence 4
If you are a conventioneer, you have instant and direct walking access to the main convention center.  

Sentence 5
But don't think about parking here for said conventions.  

Sentence 6
$30 a day for parking, Hilton?  

Sentence 7
Seriously?  

Sentence 8
That's robbery.  

Sentence 9
And greed.



Sentence 10
The biggest disappointment is the killing off of Star Trek:

Sentence 11
The Experience over two years ago.  

Sentence 12
Despite this place being off-Strip, I made it a point to go here because they had such a fun and unique attraction here.  

Sentence 13
It would be one thing if they had something in mind to replace it, but either poor planning or the economy....

Sentence 14
It's dead, Jim. 



Sentence 15
The "Space Lounge" is still there.  

Sentence 16
An entire casino area with a heavy

### Text Normalization

In [64]:
token_text = [token.orth_ for token in parsed_view]
token_lemma = [token.lemma_ for token in parsed_view]
token_shape = [token.shape_ for token in parsed_view]

pd.DataFrame(zip(token_text,token_lemma,token_shape),columns=["token_text","token_lemma","token_shape"])

,token_text,token_lemma,token_shape
0,This,this,Xxxx
1,actually,actually,xxxx
2,used,use,xxxx
3,to,to,xx
4,be,be,xx
5,one,one,xxx
6,of,of,xx
7,my,-PRON-,xx
8,favorite,favorite,xxxx
9,hotel,hotel,xxxx


### Part of Speech

In [65]:
token_pos = [token.pos_ for token in parsed_view]
pd.DataFrame(zip(token_text,token_pos),columns=["Text","POS"])

,Text,POS
0,This,DET
1,actually,ADV
2,used,VERB
3,to,PART
4,be,AUX
5,one,NUM
6,of,ADP
7,my,DET
8,favorite,ADJ
9,hotel,NOUN


In [66]:
displacy.render(parsed_view,style="ent")

In [68]:
for num, entity in enumerate(parsed_view.ents):
    print(f'Entity {num + 1}:', entity, '-', entity.label_)
    print('')

Entity 1: the day - DATE

Entity 2: 30 - MONEY

Entity 3: Hilton - ORG

Entity 4: Star Trek: - WORK_OF_ART

Entity 5: two years ago - DATE

Entity 6: Jim - PERSON

Entity 7: Star Trek - WORK_OF_ART

Entity 8: Couches - PERSON

Entity 9: Benihana - PERSON

Entity 10: Vegas - GPE

Entity 11: Hilton - ORG

Entity 12: Pizza Hut Express - ORG

Entity 13: Benihana - PRODUCT

Entity 14: Star Trek - WORK_OF_ART

Entity 15: two-plus years - DATE

Entity 16: first - ORDINAL

Entity 17: Hilton - ORG

Entity 18: one - CARDINAL

Entity 19: Vegas - GPE



In [69]:
token_entity_type = [token.ent_type_ for token in parsed_view]
token_entity_iob = [token.ent_iob_ for token in parsed_view]

pd.DataFrame(
    zip(token_text, token_entity_type, token_entity_iob),
    columns=['token_text', 'entity_type', 'inside_outside_begin']
    )

,token_text,entity_type,inside_outside_begin
0,This,,O
1,actually,,O
2,used,,O
3,to,,O
4,be,,O
5,one,,O
6,of,,O
7,my,,O
8,favorite,,O
9,hotel,,O


In [70]:
token_attributes = [
    (
        token.orth_,
        token.prob,
        token.is_stop,
        token.is_punct,
        token.is_space,
        token.like_num,
        token.is_oov,
        
    )for token in parsed_view
    
]

df = pd.DataFrame(token_attributes,columns = [
    'text',
    'log_probability',
    'stop?',
    'punctuation?',
    'space',
    'number',
    'out of vocabulary']
    )

df.loc[:,'stop?':'out of vocabulary'] = df.loc[:,'stop?':'out of vocabulary'].applymap(lambda x : x if x else "")
df

,text,log_probability,stop?,punctuation?,space,number,out of vocabulary
0,This,-20.0,True,,,,True
1,actually,-20.0,,,,,True
2,used,-20.0,True,,,,True
3,to,-20.0,True,,,,True
4,be,-20.0,True,,,,True
5,one,-20.0,True,,,True,True
6,of,-20.0,True,,,,True
7,my,-20.0,True,,,,True
8,favorite,-20.0,,,,,True
9,hotel,-20.0,,,,,True


## Phrase Modelling

In [5]:
from gensim.models.phrases import Phraser, Phrases
from gensim.models.word2vec import LineSentence

In [29]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def pronoun_lemmatize(token):
    """
    helper function to preserve pronouns and force lowercasing while lemmatizing
    """
    
    if token.lemma_ == '-PRON-':
        return token.lower_
    
    else:
        return token.lemma_.lower()

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with open(filename) as f:
        for review in f:
            yield review.replace('\\n', '\n')
    

We need to reiterate over 5Million reviews and it would take 12-14 Hours. 
<br><br>
Let's create a sample dataset out of reviews_text_all.txt

**For 100,000 Reviews it took 17min** <br>
CPU times: user 43min 46s, sys: 4min, total: 47min 46s <br>
Wall time: 16min 59s

In [10]:
review_txt_filepath_sample_path = os.path.join(scratch_path,'review_txt_file_sample.txt')

#Instead of keeping samples in one file, it will be split into 10 files of 5 Lakh(500,000) reviews each. 
#So Total Reviews will be 50 Lakhs (5 Milliom)

review_txt_filepath_sample_path1 = os.path.join(scratch_path,'review_txt_file_sample','review_txt_file_sample1.txt')
review_txt_filepath_sample_path2 = os.path.join(scratch_path,'review_txt_file_sample','review_txt_file_sample2.txt')
review_txt_filepath_sample_path3 = os.path.join(scratch_path,'review_txt_file_sample','review_txt_file_sample3.txt')
review_txt_filepath_sample_path4 = os.path.join(scratch_path,'review_txt_file_sample','review_txt_file_sample4.txt')
review_txt_filepath_sample_path5 = os.path.join(scratch_path,'review_txt_file_sample','review_txt_file_sample5.txt')
review_txt_filepath_sample_path6 = os.path.join(scratch_path,'review_txt_file_sample','review_txt_file_sample6.txt')
review_txt_filepath_sample_path7 = os.path.join(scratch_path,'review_txt_file_sample','review_txt_file_sample7.txt')
review_txt_filepath_sample_path8 = os.path.join(scratch_path,'review_txt_file_sample','review_txt_file_sample8.txt')
review_txt_filepath_sample_path9 = os.path.join(scratch_path,'review_txt_file_sample','review_txt_file_sample9.txt')
review_txt_filepath_sample_path10 = os.path.join(scratch_path,'review_txt_file_sample','review_txt_file_sample10.txt')


In [305]:
execute = False

if execute : 

    reviews_to_copy_start = 4_500_000  # Adjust Accordingly
    reviews_to_copy_end = 5_000_000 # Adjust Accordingly

    with open(review_txt_filepath_sample_path10,'w') as review_txt_filepath_sample:
        with open(review_txt_filepath) as f:
            for counter,review_line in enumerate(f):
                if reviews_to_copy_start <= counter < reviews_to_copy_end:
                    review_txt_filepath_sample.write(review_line)
else:
    reviews_to_copy_start = 0  # Adjust Accordingly
    reviews_to_copy_end = 100_000 # Adjust Accordingly

    with open(review_txt_filepath_sample_path,'w') as review_txt_filepath_sample:
        with open(review_txt_filepath) as f:
            for counter,review_line in enumerate(f):
                if reviews_to_copy_start <= counter < reviews_to_copy_end:
                    review_txt_filepath_sample.write(review_line)

In [86]:
#Let's Go

In [12]:
review_lemmatized_filepath = os.path.join(scratch_path, 'review_lemmatized_all.txt')
sentence_lemmatized_filepath = os.path.join(scratch_path, 'sentence_lemmatized_all.txt')

#Splitting the files and storing it separately

review_lemmatized_filepath1 = os.path.join(scratch_path,'review_txt_file_sample', 'review_lemmatized_all1.txt')
sentence_lemmatized_filepath1 = os.path.join(scratch_path, 'review_txt_file_sample','sentence_lemmatized_all1.txt')

In [13]:
%%time
execute = False
batch_size = 5000

if execute:
    
    with open(review_lemmatized_filepath1,'w') as review_lemmatized_file:
        with open(sentence_lemmatized_filepath1,'w') as sentence_lemmatized_file:
            
            pipe = nlp.pipe(
                line_review(review_txt_filepath_sample_path1),
                batch_size=batch_size,
                n_threads=8
            )
            
            for counter,parsed_review in enumerate(pipe):
                
                if counter%batch_size==0:
                    print(f'Iteration : {counter}')
                
                lemmatized_review = ' '.join([pronoun_lemmatize(token) for token in parsed_review if not punct_space(token)])
                
                review_lemmatized_file.write(lemmatized_review+'\n')
                
                for sentence in parsed_review.sents:
                    lemmatized_sentence = ' '.join([pronoun_lemmatize(token) for token in sentence if not punct_space(token)])
                    
                    sentence_lemmatized_file.write(lemmatized_sentence+"\n")

   

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


In [14]:
#gensim's LineSentence class provides a convenient iterator for working with other gensim components.
sentences_unigrams = LineSentence(sentence_lemmatized_filepath)

In [15]:
#Sample

for sentence_unigrams in it.islice(sentences_unigrams,60,70):
    print(' '.join(sentence_unigrams))
    print()

and i hope they continue to do well

order

in delivery review

i discover this place from the front desk of the airport hotel and they recommend it highly

i call to place my order for delivery to my room

they be very accommodating

won

ton soup nice broth and tender win ton

chicken with diced vegetable and cashew again fresh vegetable be use and good sized piece of chicken

crispy beef much like a ginger beef nice serving of beef with a sweet chilli sauce



### Bigram

In [16]:
bigram_model_filepath = os.path.join(scratch_path, 'bigram_phrase_model')

In [234]:
execute = True

if execute:

    bigram_phrases = Phrases(sentences_unigrams)
    
    # Turn the finished Phrases model into a "Phraser" object,
    # which is optimized for speed and memory use
    bigram_phrases = Phraser(bigram_phrases)
    bigram_phrases.save(bigram_model_filepath)

In [17]:
# Load the finished model

bigram_phrases = Phraser.load(bigram_model_filepath)

In [18]:
sentences_bigrams_filepath = os.path.join(scratch_path,'sentence_bigram_phrases_all.txt')

In [238]:
execute = True

if execute:
    with open(sentences_bigrams_filepath,'w') as f:
        
        for sentence_unigrams in sentences_unigrams:
            sentence_bigrams = ' '.join(bigram_phrases[sentence_unigrams])
            f.write(sentence_bigrams+'\n')

In [19]:
sentences_bigrams = LineSentence(sentences_bigrams_filepath)

In [20]:
for sentence_bigrams in it.islice(sentences_bigrams, 60, 70):
    print(' '.join(sentence_bigrams))
    print('')

and i hope they continue to do well

order

in delivery review

i discover this place from the front_desk of the airport hotel and they recommend it highly

i call to place my order for delivery to my room

they be very accommodating

won

ton soup nice broth and tender win_ton

chicken with diced vegetable and cashew again fresh vegetable be use and good sized piece of chicken

crispy beef much like a ginger beef nice serving of beef with a sweet chilli sauce



In [21]:
trigram_model_filepath = os.path.join(scratch_path, 'trigram_phrase_model')

In [242]:
# this is a bit time consuming - set execute = True
# if you want to execute modeling yourself.

execute = True

if execute:

    trigram_phrases = Phrases(sentences_bigrams)
    
    # Turn the finished Phrases model into a "Phraser" object,
    # which is optimized for speed and memory use
    trigram_phrases = Phraser(trigram_phrases)
    trigram_phrases.save(trigram_model_filepath)

In [22]:
# load the finished model from disk
trigram_phrases = Phraser.load(trigram_model_filepath)

In [23]:
sentences_trigrams_filepath = os.path.join(scratch_path, 'sentence_trigram_phrases_all.txt')

In [246]:
# this is a bit time consuming - set execute = True
# if you want to execute data prep yourself.

execute = True

if execute:

    with open(sentences_trigrams_filepath, 'w') as f:
        
        for sentence_bigrams in sentences_bigrams:
            
            sentence_trigrams = ' '.join(trigram_phrases[sentence_bigrams])
            
            f.write(sentence_trigrams + '\n')

In [24]:
sentences_trigrams = LineSentence(sentences_trigrams_filepath)

In [25]:
for sentence_trigrams in it.islice(sentences_trigrams, 20, 30):
    print(' '.join(sentence_trigrams))
    print('')

my only_gripe be the one fella with red hair

he make the round while clean but no smile or personality a few night ago he toss the napkin i just put on the table to help go with my meal

after i be do he just reach for my tray no excuse me or be you do with that

i realize he be try to do his job quickly but a little table manner go along way

that be say still like to grub here and glad that there be finally a cane_'s close to me

this be definitely my favorite fast food sub_shop

ingredient be everything and everything i see and taste here tell me that they be use top grade fresh ingredient

the brisket sandwich be probably my favorite and it be the one my wife always get

unlike her i often bounce_around the menu to try different thing

definitely a step up from subway_quiznos jimmy_johns etc in my opinion



In [28]:
review_trigrams_filepath = os.path.join(scratch_path, 'review_trigrams_all.txt')

In [250]:

execute = True

if execute:
    
    reviews_lemmatized = LineSentence(review_lemmatized_filepath)

    with open(review_trigrams_filepath, 'w') as f:
        
        for review_unigrams in reviews_lemmatized:
                        
            # apply the first-order and second-order phrase models
            review_bigrams = bigram_phrases[review_unigrams]
            review_trigrams = trigram_phrases[review_bigrams]

            # remove any remaining stopwords
            review_trigrams = [
                term
                for term in review_trigrams
                if term not in nlp.Defaults.stop_words
                ]

            # write the transformed review as a line in the new file
            review_trigrams = ' '.join(review_trigrams)
            f.write(review_trigrams + '\n')

In [30]:
review_num = 22

print('Original:' + '\n')

for review in it.islice(line_review(review_txt_filepath), review_num, review_num+1):
    print(review)

print('----' + '\n')
print('Transformed:' + '\n')

with open(review_trigrams_filepath) as f:
    for review in it.islice(f, review_num, review_num+1):
        print(review)

Original:

This actually used to be one of my favorite hotel/casinos back in the day.  Oh, how things have changed.

There are some goods, however.  If you are a conventioneer, you have instant and direct walking access to the main convention center.  But don't think about parking here for said conventions.  $30 a day for parking, Hilton?  Seriously?  That's robbery.  And greed.

The biggest disappointment is the killing off of Star Trek: The Experience over two years ago.  Despite this place being off-Strip, I made it a point to go here because they had such a fun and unique attraction here.  It would be one thing if they had something in mind to replace it, but either poor planning or the economy.... It's dead, Jim. 

The "Space Lounge" is still there.  An entire casino area with a heavy sci-fi theme to it.  Except there's no casino here.  It's a huge space themed area with a ghost town bar, no slots, a few couches, a makeshift stage, beer signs, and a few remnants of Star Trek theme

## Topic Modelling using LDA

In [31]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle

In [32]:
dictionary_filepath = os.path.join(scratch_path,'trigram_dict_all.dict')

In [253]:
execute = False

if execute:

    reviews_trigrams = LineSentence(review_trigrams_filepath)

    # learn the dictionary by iterating over all of the reviews
    dictionary_trigrams = Dictionary(reviews_trigrams)
    
    # filter tokens that are very rare or too common from
    # the dictionary (filter_extremes) and reassign integer ids (compactify)
    dictionary_trigrams.filter_extremes(no_below=20, no_above=0.4)
    dictionary_trigrams.compactify()

    dictionary_trigrams.save(dictionary_filepath)

In [33]:
dictionary_trigrams = Dictionary.load(dictionary_filepath)

## Bag of Words

In [34]:
bow_corpus_filepath = os.path.join(scratch_path,'bow_trigrams_corpus_all.mm')

In [35]:
def bow_generator(filepath):
    
    for review in LineSentence(filepath):
        yield dictionary_trigrams.doc2bow(review)

In [257]:
execute = False

if execute:
    MmCorpus.serialize(
        bow_corpus_filepath,
        bow_generator(review_trigrams_filepath)
    )

In [36]:
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(bow_corpus_filepath)

In [37]:
lda_model_filepath = os.path.join(scratch_path, 'lda_model_all')

In [261]:
execute = False

if execute:
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
    lda = LdaMulticore(
        trigram_bow_corpus,
        num_topics=20,
        id2word=dictionary_trigrams,
        workers=7
    )
    
    lda.save(lda_model_filepath)

In [38]:
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

In [39]:
def explore_topic(topic_number, topn=25):
        
    print(f'{"term":20} {"frequency"}' + '\n')

    for term, frequency in lda.show_topic(topic_number, topn=25):
        print(f'{term:20} {round(frequency, 3):.3f}')

In [40]:
#10,15 bar 19 Japanese 9. chicken, 16 pizza/bread, 18 chinese,
explore_topic(topic_number=0)

term                 frequency

great                0.028
service              0.018
restaurant           0.016
delicious            0.010
amazing              0.009
like                 0.009
excellent            0.008
friendly             0.007
staff                0.007
love                 0.007
definitely           0.007
nice                 0.007
come                 0.007
atmosphere           0.006
steak                0.006
menu                 0.006
price                0.006
little               0.006
time                 0.006
enjoy                0.005
order                0.005
dish                 0.005
visit                0.005
experience           0.005
try                  0.005


In [41]:
topic_names = {
    0:'Experience_Good',
    1:'Dinner_Drinks',
    2:'Pizza',
    3:'Waffle',
    4:'Vietnam',
    5:'Ice_Cream',
    6:'Indian',
    7:'Thai',
    8:'Experience 1',
    9:'Chicken',
    10:'Bar',
    11:'Caffe',
    12:'Shrimp',
    13:'Positive',
    14:'Korean',
    15:'Bar',
    16:'Italian',
    17:'Experience 2',
    18:'Chinese',
    19:'Japanese'
    
}

In [323]:
topic_names_filepath = os.path.join(scratch_path,'topic_names.pkl')

with open(topic_names_filepath,'wb') as f:
    pickle.dump(topic_names,f)

In [44]:
LDAvis_data_filepath = os.path.join(scratch_path,'ldavis_prepared')

In [45]:
execute = False

if execute:
    
    LDAvis_prepared = pyLDAvis.gensim.prepare(
            lda,
            trigram_bow_corpus,
            dictionary_trigrams
    )
    
    with open(LDAvis_data_filepath,'wb') as f:
        pickle.dump(LDAvis_prepared,f)

In [46]:
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

In [47]:
pyLDAvis.display(LDAvis_prepared)

Unfortunately, though the data used by gensim and pyLDAvis are the same, they don't use the same ID numbers for topics. If you need to match up topics in gensim's LdaMulticore object and pyLDAvis' visualization, you have to dig through the terms manually.

## Describing text with LDA

In [48]:
def get_sample_review(review_number):
    
    review = next(
        it.islice(
            line_review(review_txt_filepath_sample_path),
            review_number,
            review_number+1
        )
    )
    
    return review
    

In [49]:
def lda_description(review_text, min_topic_freq=0.05):
    
    # parse the review text with spaCy
    parsed_review = nlp(review_text)
    
    # lemmatize the text and remove punctuation and whitespace
    review_unigrams = [
        pronoun_lemmatize(token)
        for token in parsed_review
        if not punct_space(token)
        ]
    
    # apply the first-order and secord-order phrase models
    review_bigrams = bigram_phrases[review_unigrams]
    review_trigrams = trigram_phrases[review_bigrams]
    
    # remove any remaining stopwords
    review_trigrams = [
        term
        for term in review_trigrams
        if not term in nlp.Defaults.stop_words
        ]
    
    # create a bag-of-words representation
    review_bow = dictionary_trigrams.doc2bow(review_trigrams)
    
    # create an LDA representation
    review_lda = lda[review_bow]
    
    # sort with the most highly related topics first
    review_lda = sorted(review_lda, key=lambda topic_number_freq: -topic_number_freq[-1])
    
    for topic_number, freq in review_lda:
        if freq < min_topic_freq:
            break
            
        # print the most highly related topic names and frequencies
        print(f'{topic_names[topic_number]:25} {round(freq, 3):.3f}')

In [50]:
sample_review = get_sample_review(0)

print(sample_review)

I love Deagan's. I do. I really do. The atmosphere is cozy and festive. The shrimp tacos and house fries are my standbys. The fries are sometimes good and sometimes great, and the spicy dipping sauce they come with is to die for. The beer list is amazing and the cocktails are great. The prices are mid-level, so it's not a cheap dive you can go to every week, but rather a treat when you do. Try it out. You won't be disappointed!



In [51]:
lda_description(sample_review)

Experience_Good           0.457
Bar                       0.386
Chicken                   0.077
Korean                    0.053


In [52]:
sample_review = get_sample_review(3)

print(sample_review)

This is definitely my favorite fast food sub shop. Ingredients are everything, and everything I see and taste here tells me that they're using top-grade fresh ingredients. The brisket sandwich is probably my favorite... and it's the one my wife ALWAYS gets. Unlike her, I often bounce around the menu to try different things. Definitely a step up from Subway, Quiznos, Jimmy Johns, etc in my opinion. As with all of my reviews, I grade each place relative to what I perceive to be its peers - so five star compared to them.



In [53]:
lda_description(sample_review)

Bar                       0.401
Vietnam                   0.369
Positive                  0.206


## Word2vec

In [56]:
from gensim.models import Word2Vec

sentences_trigrams = LineSentence(sentences_trigrams_filepath)
word2vec_filepath = os.path.join(scratch_path, 'word2vec_model_all')


In [57]:

execute = False

if execute:
    food2vec = Word2Vec(
        sentences_trigrams,
        size=100,
        window=5,
        min_count=50,
        sg=1,
        workers=7,
        iter=20
        
    )
    
    food2vec.save(word2vec_filepath)

In [58]:
food2vec = Word2Vec.load(word2vec_filepath)
food2vec.init_sims()

print(f'{food2vec.epochs} training epochs so far.')

20 training epochs so far.


In [59]:
print(f'{len(food2vec.wv.vocab):,} terms in the food2vec vocabulary.')

13,936 terms in the food2vec vocabulary.


In [60]:
ordered_vocab = [
    (term, voc.index, voc.count)
    for term, voc in food2vec.wv.vocab.items()
    ]

# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda term_tuple: -term_tuple[2])

# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# create a DataFrame with the food2vec vectors as data,
# and the terms as row labels
word_vectors = pd.DataFrame(
    food2vec.wv.vectors_norm[term_indices, :],
    index=ordered_terms
    )

word_vectors

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
be,-0.057964,0.093024,-0.056910,-0.108527,-0.112589,0.062562,0.113427,0.087544,0.019582,0.146057,...,-0.129514,0.226504,-0.024057,-0.001877,-0.014504,-0.213367,-0.013456,-0.044289,-0.093961,-0.113130
the,0.048503,0.217682,-0.005025,-0.050573,-0.028011,0.130259,0.072993,-0.055588,0.069489,0.039797,...,0.093999,0.167828,0.186657,0.198357,0.007146,-0.137790,-0.074278,0.033046,0.101660,-0.176706
and,0.151287,0.152073,-0.055823,0.182969,-0.095177,0.173844,0.035760,-0.023735,0.095828,0.076436,...,-0.158770,0.176054,-0.057037,0.193354,-0.074707,-0.019295,0.046843,-0.040015,0.158411,-0.078967
i,-0.069010,0.056788,-0.125057,0.169204,-0.025230,0.090932,0.047315,0.148966,-0.012014,0.019668,...,-0.080860,-0.015290,0.125934,-0.053920,0.075860,-0.073164,-0.102483,0.015971,0.163588,-0.150244
a,0.055974,0.097769,-0.148451,0.084065,-0.066769,0.216107,0.102495,-0.077068,0.101502,-0.054228,...,-0.069064,0.012306,-0.083945,0.015771,-0.019746,-0.007172,-0.093071,-0.012525,-0.020476,-0.221641
to,-0.143566,0.031047,-0.110245,0.032759,-0.095907,0.018251,-0.106276,0.113439,0.155031,-0.017730,...,-0.032690,-0.029109,0.031127,0.084714,0.137827,0.065837,0.018692,0.061278,0.098118,-0.095441
it,-0.026759,-0.048544,-0.112626,-0.045125,-0.057062,0.214846,0.088156,0.178657,0.048712,0.099787,...,0.046854,0.028191,0.158214,-0.144276,-0.022272,-0.188373,0.038064,-0.156452,0.094970,-0.056367
have,0.052764,-0.035009,-0.148822,-0.041219,0.167187,0.262548,0.110669,0.008106,0.115148,0.164899,...,-0.156289,0.140474,0.081975,0.081694,-0.073792,-0.141091,-0.154519,0.161871,-0.060003,0.025035
of,-0.029022,0.159614,0.080510,-0.001179,0.072259,0.127428,-0.075180,-0.008125,-0.066882,-0.039957,...,0.034715,-0.089672,-0.108513,-0.023947,-0.051213,-0.113778,0.130575,0.140443,0.079844,-0.170224
not,-0.078675,0.029845,-0.105665,-0.057722,0.209949,0.051736,0.153405,0.144148,0.096614,-0.069805,...,0.082152,-0.014057,-0.016794,0.046649,-0.082164,-0.151225,-0.001193,-0.116331,0.154199,-0.026757


In [61]:
def get_related_terms(token, topn=10):

    for word, similarity in food2vec.wv.most_similar(positive=[token], topn=topn):

        print(f'{word:20} {round(similarity, 3)}')

In [62]:
get_related_terms("dal")

masala               0.779
korma                0.759
daal                 0.738
curry                0.72
tikka_masala         0.714
paneer               0.705
sambar               0.704
chicken_tikka_masala 0.703
pakora               0.697
samosas              0.694


In [63]:
get_related_terms('happy_hour', topn=15)

hh                   0.899
reverse_happy_hour   0.805
during_happy_hour    0.788
happy_hr             0.702
taco_tuesday         0.692
m_f                  0.677
taco_tuesdays        0.662
frozen_cosmo         0.638
wednesday            0.636
mon_fri              0.635
thursday             0.631
6_p.m.               0.62
monday               0.614
6pm                  0.613
8pm                  0.611


In [368]:
get_related_terms('pasta', topn=15)

spaghetti            0.821
penne                0.818
lasagna              0.798
gnocchi              0.796
linguine             0.789
fettuccine           0.785
carbonara            0.767
pomodoro             0.762
pastas               0.761
tortellini           0.761
fettuccini           0.759
linguini             0.753
bolognese            0.746
tagliatelle          0.745
alfredo              0.743


## Word Algebra

In [64]:
def word_algebra(add=[], subtract=[], topn=1):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = food2vec.wv.most_similar(positive=add, negative=subtract, topn=topn)
    
    for term, similarity in answers:
        print(term)

In [65]:
word_algebra(add=['lunch', 'night'], subtract=['day'])

dinner


In [66]:
word_algebra(add=['breakfast', 'lunch'], topn=2)

brunch
sunday_brunch


In [67]:
word_algebra(add=["meat", 'masala'],subtract=['tikka'],topn=2)

vegetable
protein


In [68]:
word_algebra(add=['coffee', 'snack'], subtract=['drink'])

pastry


## Word Vector Visualization with t-SNE

t-SNE = t distributed Stochastic Neighbor Embedding

In [70]:
from sklearn.manifold import TSNE

In [71]:
tsne_input = (
word_vectors.drop(nlp.Defaults.stop_words,errors='ignore').head(5000)
)

tsne_input.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
good,-0.043445,0.042326,-0.110177,0.035527,0.151720,0.174476,0.046227,0.047874,0.039682,0.231888,...,-0.062330,-0.054948,0.055228,0.183794,0.081507,-0.160624,-0.072325,-0.114533,-0.002273,-0.027644
food,0.055091,-0.027724,-0.019536,-0.134537,-0.033250,0.090470,0.087247,0.234323,0.046579,0.203205,...,-0.009469,-0.101045,-0.132566,0.144980,0.155957,-0.215573,0.063888,-0.056524,0.182774,-0.039282
place,0.016968,-0.167146,-0.058808,-0.031546,0.073444,0.005827,0.081423,0.256721,0.207352,0.167439,...,-0.206446,0.003490,-0.063638,-0.100111,0.101359,-0.154605,0.035992,0.015832,-0.082489,-0.146338
order,-0.041543,-0.071888,-0.121205,0.057029,0.036558,0.130947,-0.245544,-0.064761,0.078767,0.024599,...,-0.021817,0.180065,-0.120688,0.116845,-0.119522,0.017506,-0.016650,-0.037909,0.026007,-0.176749
great,0.126145,0.126297,-0.186980,0.089585,0.062044,0.114617,-0.046565,-0.048413,-0.002841,0.304777,...,-0.043893,-0.002471,-0.046111,0.231201,0.019373,-0.147272,-0.063334,-0.103539,0.001218,0.056041


In [72]:
tsne_filepath = os.path.join(scratch_path, 'tsne_model')

tsne_vectors_filepath = os.path.join(scratch_path, 'tsne_vectors.npy')

In [73]:
execute = True

if execute:
    tsne = TSNE()
    tsne_vectors = tsne.fit_transform(tsne_input)
    
    with open(tsne_filepath,'wb') as f:
        pickle.dump(tsne,f)
        
    pd.np.save(tsne_vectors_filepath, tsne_vectors)

In [74]:
with open(tsne_filepath,'rb') as f:
    tsne = pickle.load(f)
    
tsne_vectors = pd.np.load(tsne_vectors_filepath)

tsne_vectors = pd.DataFrame(
    tsne_vectors,
    index=pd.Index(tsne_input.index),
    columns=['x_coord', 'y_coord']
    )

In [75]:
tsne_vectors.head()

,x_coord,y_coord
good,4.970875,-29.925600
food,-18.185455,2.356030
place,-46.668331,7.702044
order,-31.440905,-30.983326
great,4.008860,-26.430378


In [76]:
tsne_vectors['word'] = tsne_vectors.index


## Plotting Word Vectors with Bokeh

In [77]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

output_notebook()

/Users/tendulkar/anaconda3/lib/python3.7/site-packages/bokeh/plotting/helpers.py:21: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, OrderedDict, Sequence


Loading BokehJS ...

In [78]:
plot_data=ColumnDataSource(tsne_vectors)

tsne_plot = figure(
    title='t-SNE Word Embeddings',
    plot_width=800,
    plot_height=800,
    tools=(
        'pan, wheel_zoom, box_zoom,'
        'box_select, reset'
        ),
    active_scroll='wheel_zoom'
    )

tsne_plot.add_tools(
    HoverTool(tooltips = '@word')
    )

tsne_plot.circle(
    'x_coord',
    'y_coord',
    source=plot_data,
    color='blue',
    line_alpha=0.2,
    fill_alpha=0.1,
    size=10,
    hover_line_color='black'
    )

tsne_plot.title.text_font_size = value('16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# engage!
show(tsne_plot);

/Users/tendulkar/anaconda3/lib/python3.7/site-packages/bokeh/core/property/container.py:102: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(value, (collections.Container, collections.Sized, collections.Iterable))
/Users/tendulkar/anaconda3/lib/python3.7/site-packages/bokeh/core/property/container.py:102: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(value, (collections.Container, collections.Sized, collections.Iterable))


## Text Categorization with spacy textcat

In [80]:
with open(review_path) as f:
    first_review=next(f)
    print(first_review)
    print(json.loads(first_review))

{"review_id":"xQY8N_XvtGbearJ5X4QryQ","user_id":"OwjRMXRC0KyPrIlcjaXeFQ","business_id":"-MhfebM0QIsKt87iDN-FNw","stars":2.0,"useful":5,"funny":0,"cool":0,"text":"As someone who has worked with many museums, I was eager to visit this gallery on my most recent trip to Las Vegas. When I saw they would be showing infamous eggs of the House of Faberge from the Virginia Museum of Fine Arts (VMFA), I knew I had to go!\n\nTucked away near the gelateria and the garden, the Gallery is pretty much hidden from view. It's what real estate agents would call \"cozy\" or \"charming\" - basically any euphemism for small.\n\nThat being said, you can still see wonderful art at a gallery of any size, so why the two *s you ask? Let me tell you:\n\n* pricing for this, while relatively inexpensive for a Las Vegas attraction, is completely over the top. For the space and the amount of art you can fit in there, it is a bit much.\n* it's not kid friendly at all. Seriously, don't bring them.\n* the security is n

In [81]:
total_examples = 1000

funny_reviews = []
unfunny_reviews = []

with open(review_path) as f:
    for idx, review in enumerate(f):
        
        review = json.loads(review)
        
        if review['funny'] > 5 and len(funny_reviews) < (total_examples / 2):
           
            funny_reviews.append((
                review['text'], {
                    'cats': {
                        'FUNNY': 1.0,
                        'UNFUNNY': 0.0
                        }
                    }
            ))
            
            continue
            
        if review['funny'] == 0 and len(unfunny_reviews) < (total_examples / 2):
            
            unfunny_reviews.append((
                review['text'], {
                    'cats': {
                        'FUNNY': 0.0,
                        'UNFUNNY': 1.0
                        }
                    }
            ))
            
            continue
            
        if len(funny_reviews) >= (total_examples / 2) and len(unfunny_reviews) >= (total_examples / 2):
            
            break


In [82]:
review_text,review_cats = funny_reviews[1]

print(review_text)
print(review_cats)

I normally don't bother with dealing health stores but I have wanted to try out a hydrogen peroxide therapy to help detox the body and I didn't want to wait 5 days for shipping from amazon and this place popped up.

It was relatively close to where I was and whole foods doesn't carry the 35% h2o2 so I stopped by here. Even though the parking was horrible, the store was pretty open and I was greeted by the owner (or I think it was the owner) and he was very friendly.

I asked for my 35% food grade peroxide and he had some, and it was way cheaper than paying for it on amazon.

5 stars for being clutch, and for having cheap prices. 

This is a mom and pop store, check them out if you are in the area.
{'cats': {'FUNNY': 1.0, 'UNFUNNY': 0.0}}


In [83]:
review_text,review_cats = unfunny_reviews[1]

print(review_text)
print(review_cats)

I love Deagan's. I do. I really do. The atmosphere is cozy and festive. The shrimp tacos and house fries are my standbys. The fries are sometimes good and sometimes great, and the spicy dipping sauce they come with is to die for. The beer list is amazing and the cocktails are great. The prices are mid-level, so it's not a cheap dive you can go to every week, but rather a treat when you do. Try it out. You won't be disappointed!
{'cats': {'FUNNY': 0.0, 'UNFUNNY': 1.0}}


In [84]:
import random

In [86]:
train_data = funny_reviews[:int(len(funny_reviews)/2)] + unfunny_reviews[:int(len(funny_reviews)/2)]
test_data = funny_reviews[int(len(funny_reviews)/2):] + unfunny_reviews[int(len(funny_reviews)/2):]

random.shuffle(train_data)
random.shuffle(test_data)

In [87]:
#Lets create a new pipleine and add it to existing one
original_pipe_names = nlp.pipe_names
original_pipe_names

['tagger', 'parser', 'ner']

In [88]:
textcat = nlp.create_pipe('textcat', config={'exclusive_classes':True})

textcat.add_label('FUNNY')
textcat.add_label("UNFUNNY")

1

In [89]:
nlp.add_pipe(textcat)
nlp.pipe_names

['tagger', 'parser', 'ner', 'textcat']

In [90]:
def evaluate(tokenizer, textcat, texts, cats, label='FUNNY'):
    
    docs = (tokenizer(text) for text in texts)
    
    tp = 0.0  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 0.0  # True negatives
    
    for i, doc in enumerate(textcat.pipe(docs)):

        gold = cats[i]['cats']
        score = doc.cats[label]
        
        if label not in gold:
            continue
        if score >= 0.5 and gold[label] >= 0.5:
            tp += 1.0
        elif score >= 0.5 and gold[label] < 0.5:
            fp += 1.0
        elif score < 0.5 and gold[label] < 0.5:
            tn += 1
        elif score < 0.5 and gold[label] >= 0.5:
            fn += 1
    
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    
    if (precision + recall) == 0:
        f_score = 0.0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)
    
    return {"textcat_p": precision, "textcat_r": recall, "textcat_f": f_score}

In [91]:
from spacy.util import minibatch

In [92]:
%%time

with nlp.disable_pipes(*original_pipe_names):
    
    optimizer = nlp.begin_training()
    
    print("Training the model...")
    print("{:^5}\t{:^5}\t{:^5}\t{:^5}".format("LOSS", "P", "R", "F"))
    
    for i in range(10):
        losses = {}

        
        random.shuffle(train_data)

        batches = minibatch(train_data, size=8)

        for batch in batches:
            texts, cats = zip(*batch)
            nlp.update(texts, cats, sgd=optimizer, drop=0.2, losses=losses)

        with textcat.model.use_params(optimizer.averages):

            
            test_texts, test_cats = zip(*test_data)
            scores = evaluate(nlp.tokenizer, textcat, test_texts, test_cats)

        print(
            "{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}".format(  
                losses["textcat"],
                scores["textcat_p"],
                scores["textcat_r"],
                scores["textcat_f"],
                )
            )

Training the model...
LOSS 	  P  	  R  	  F  
0.511	0.601	0.844	0.702
0.454	0.675	0.824	0.742
0.352	0.692	0.764	0.726
0.283	0.729	0.744	0.737
0.207	0.711	0.680	0.695
0.185	0.739	0.680	0.708
0.142	0.719	0.716	0.717
0.096	0.721	0.692	0.706
0.110	0.728	0.684	0.705
0.065	0.726	0.688	0.706
CPU times: user 2min 34s, sys: 6.9 s, total: 2min 41s
Wall time: 40.9 s
